In [1]:
import requests
import sys
import os
import urllib3
import bs4 as bs
import re
import pandas as pd

In [2]:
with open("arks.txt", "r") as f:
    zelist = f.readlines()

In [3]:
mylist = []
for i in zelist:
    mylist.append(re.sub("\\n","", i))
    

In [4]:
len(mylist)

230

In [5]:
def metadata_collector(list_ark):
    '''
    Retrieves the metadata of a newspaper issue from Gallica.
    '''
    arks = []
    date = []
    year = []
    issue = []
    ident = []
    for ark in list_ark:
        print(ark)
        arks.append(ark)
        http = urllib3.PoolManager()
        request = http.request('GET', "https://gallica.bnf.fr/services/OAIRecord?ark=" + ark)
        xml = request.data
        with open("data.xml", 'wb') as f:
            f.write(xml)
        with open("data.xml", 'r') as f:
            content = f.read()
        soup = bs.BeautifulSoup(content, "xml")
        for tag in soup.find('dc:date'):
            date.append(tag.get_text())
        for tag in soup.find('dc:identifier'):
            ident.append(tag.get_text())
        for tag in soup.find_all('dc:description'):
            t = tag.get_text()
            if re.search(r"A(\d+),N(\d{3})", t):
                year.append(re.search(r"A(\d+),N(\d{3})", t).group(1))
                issue.append(re.search(r"A(\d+),N(\d{3})", t).group(2))
    return arks, date, year, issue, ident

In [6]:
arks, dates, years, issues, idents = metadata_collector(mylist)

bpt6k1330952j
bpt6k1330806z
bpt6k1330890w
bpt6k1330798v
bpt6k1330808s
bpt6k1330986b
bpt6k1330823g
bpt6k13307805
bpt6k13307857
bpt6k1330879v
bpt6k1331008b
bpt6k1330963b
bpt6k1330833v
bpt6k13308541
bpt6k13308400
bpt6k1330849q
bpt6k1330883r
bpt6k1330784t
bpt6k1330874s
bpt6k1330978s
bpt6k1330923t
bpt6k13310127
bpt6k13308029
bpt6k1330830m
bpt6k13309262
bpt6k13489277
bpt6k13308133
bpt6k1330976z
bpt6k13307783
bpt6k13309492
bpt6k1330795m
bpt6k1330996q
bpt6k13309247
bpt6k13308704
bpt6k1330788g
bpt6k1330842t
bpt6k1330779h
bpt6k1331011t
bpt6k13307820
bpt6k1330980v
bpt6k13307961
bpt6k13309818
bpt6k13308934
bpt6k13309655
bpt6k1330815x
bpt6k1330968d
bpt6k13309017
bpt6k1330781k
bpt6k13308118
bpt6k1330895z
bpt6k1330920k
bpt6k13309425
bpt6k1330801w
bpt6k1330907q
bpt6k1330828j
bpt6k1330841d
bpt6k13308630
bpt6k1330987r
bpt6k1330899m
bpt6k13309121
bpt6k1330814h
bpt6k13308593
bpt6k1330824w
bpt6k1330964r
bpt6k1330930z
bpt6k1330984h
bpt6k1330805j
bpt6k1330812p
bpt6k13310053
bpt6k1330786n
bpt6k1330876m
bpt6k1

In [7]:
df = pd.DataFrame({"Ark":arks, "Date":dates, "Year":years, "Issue":issues, "Identifier":idents})

In [8]:
df

,Ark,Date,Year,Issue,Identifier
0,bpt6k1330952j,1930-06-01,11,402,https://gallica.bnf.fr/ark:/12148/bpt6k1330952j
1,bpt6k1330806z,1925-07-26,6,213,https://gallica.bnf.fr/ark:/12148/bpt6k1330806z
2,bpt6k1330890w,1928-12-15,9,338,https://gallica.bnf.fr/ark:/12148/bpt6k1330890w
3,bpt6k1330798v,1925-05-31,6,205,https://gallica.bnf.fr/ark:/12148/bpt6k1330798v
4,bpt6k1330808s,1925-08-16,6,215,https://gallica.bnf.fr/ark:/12148/bpt6k1330808s
...,...,...,...,...,...
225,bpt6k13308437,1927-06-10,8,290,https://gallica.bnf.fr/ark:/12148/bpt6k13308437
226,bpt6k13308986,1929-03-03,10,347,https://gallica.bnf.fr/ark:/12148/bpt6k13308986
227,bpt6k1330885k,1928-10-20,9,333,https://gallica.bnf.fr/ark:/12148/bpt6k1330885k
228,bpt6k13308452,1927-07-01,8,292,https://gallica.bnf.fr/ark:/12148/bpt6k13308452


In [9]:
df.to_csv("metadata.tsv", "\t")